# Functions and Structs

## Topics
- Interaction between functions and types
  - Duck-typing
  - Multiple dispatch
- Defining new types (structs)
- Mutating vs. non-mutating functions
- Unmutable and mutable structs

## Functions in a functional language
Functions are the building blocks of Julia code, acting as the subroutines, procedures, blocks, and similar structural concepts found in other programming languages.

- A function's job is to take a tuple of values as an argument list and return a value. 
- If the arguments contain mutable values like arrays, the array can be modified inside the function. 
    - By convention, an exclamation mark (!) at the end of a function's name indicates that the function may modify its arguments.

## How to declare a function

Julia gives us a few different ways to write a function. The first (and most standard) requires `function` and `end` keywords.

In [ ]:
function sayhi(name)
    println("Hi $name, nice to meet you!")
end

sayhi("R2D2")

In [ ]:
function f(x)
    return x^2
end

f(42)

## Exercise 1: Addone

Write a function named `addone` that adds 1 to its input.

## Exercise 2: Polynomial

Write your own 2nd order polynomial function `poly2(x)` that evaluates $4 + 3x + 2x^2$.

Expand the function to take the polynomial coefficients as a parameter.

The function should take two parameters, `x` and `coeffs`. `coeffs` should be an array of length `3` that holds the coefficients of the polynomial. Internally your function should then compute $C_1 + C_2 x + C_3 x^2$ where $C_i$ is the $i$th element of the `coeff` array.

### Single line function definitions
Alternatively, we could have spared a few lines of code and written:

In [ ]:
sayhi2(name) = println("Hi $name, nice to meet you!")

In [ ]:
f2(x) = x^2

### Anonymous functions

Anonymous functions are nice when, for example, passing a function as a parameter:

In [ ]:
x -> x^2

In [ ]:
function functional_square(f, x)
    f(f(x))
end

functional_square(x -> x^2, 2)

## Duck-typing in Julia
*"If it quacks like a duck, it's a duck."*

Julia functions will just work on whatever input makes sense. For example, `sayhi` works also with the name written as an integer:

In [ ]:
sayhi(55595472)

And `f` will work on a matrix

In [ ]:
A = rand(3,3)

In [ ]:
f(A)

## Mutating vs. non-mutating functions
By convention, functions followed by `!` alter their contents and functions lacking `!` do not. This however, is just convention and changing the name of a function by adding `!` does not alter the function.

For example, let's look at the difference between `sort` and `sort!`.

In [ ]:
v = [3,5,2]

In [ ]:
sort(v)

In [ ]:
v

`sort(v)` returns a sorted array that contains the same elements as `v`, but `v` is left unchanged.

On the other hand, when we run `sort!(v)` the content of `v` is really modified.

In [ ]:
sort!(v)

In [ ]:
v

## Exercise 3: push?

Earlier we used a function called push!. Is there a function called push (without exclamation mark)?

## Structs

Structs are the way you declare custom data types in Julia. Essentially, a struct wraps one or more other datatypes inside and gives it a new name.

This is already a useful concept, but it gets much more powerful when we talk about functions and multiple dispatch.

You define a struct like this:

In [ ]:
struct MyInt
    number::Integer
end

So MyInt contains a single integer. You construct a new MyInt type variable by calling MyInt:

In [ ]:
myint = MyInt(5)

You can also now access the content of myint directly:

In [ ]:
myint.number

### Mutable

Notice that we cannot assing directly to `myint.number`:

In [ ]:
myint.number = 3

Structs are immutable by default. To change this, you can add the keyword `mutable`.

In [ ]:
mutable struct MyMutableInt
    number::Integer
end

mymutableint = MyMutableInt(4)
println(mymutableint)

mymutableint.number = 3
println(mymutableint)

### @enum

Here is a quick way of creating a new type with a small number of possible values.

In [ ]:
@enum Fruit orange banana apple pineaple lemon

Slide Type
The new type is called `Fruit`. An object of type Fruit can only be one of the listed things.
​
Enum is short for enumerate. This is because, under the hood, each name is represented by a number.

In [ ]:
myfruit = apple
myfruit

In [ ]:
typeof(myfruit)

## Epidemic Simulation

In the epidemic simulation, the board consists of individual plants in one of several states. 

On each time step, every cell will interact with each of it's neighbours. If the neighbour is infected, there is a change this one will become infected as well.


First, let's create a datatype for a single cell. This needs to be mutable, since the cell can become infected and recover. We'll use `@enum` to list possible states and add a second variable, `infection_time`, to measure how long the cell has been infected.

In [ ]:
"Enumerate possible states of a single cell"
@enum InfectionStatus uninfected infected dead recovered

"Data structure containing the infection status of a cell"
mutable struct Cell
    status::InfectionStatus
    infection_time::Int8
end

In [ ]:
# We can create a cell in any status
cell1 = Cell(uninfected, 0)
cell2 = Cell(infected, 0)

In [ ]:
# And we can change the infection status
cell1.status = infected

### Interaction

Next we'll need to implement the interaction between cells. Since we need this code later, we will wrap it in a function. If the other cell is infected, there is a change this one becomes infected as well.

Here we use a common trick to simulate a probability: if a random number between 1 and 0 is smaller than the probability, the cell gets infected.

In [ ]:
"""Simulate an interaction between two cells. In the other cell is
   infected, it may infect the this cell.
"""
function interact!(this_cell::Cell, other_cell::Cell, infection_rate)
    if this_cell.status == uninfected && other_cell.status == infected
        if rand(1)[1] < infection_rate
            this_cell.status = infected
            this_cell.infection_time = 0
        end
    end
end

In [ ]:
println(cell1)
interact!(cell1, cell2, 0.5)
println(cell1)

## Multiple dispatch
Until now, we have, in our examples, defined only functions with a single method having unconstrained argument types. 

Such functions behave just like they would in traditional dynamically typed languages. Nevertheless, we have used multiple dispatch and methods almost continually without being aware of it: all of Julia's standard functions and operators have many methods defining their behavior over various possible combinations of argument type and count.

This is known as multiple dispatch!

In [ ]:
function myfunc(x::Float64, y::Float64)
    2x + y
end

In [ ]:
myfunc(2,2)

In [ ]:
myfunc(2.0,2.0)

The arguments must be precisely of type `Float64`. Other numeric types, such as integers or 32-bit floating-point values, are not automatically converted to 64-bit floating-point, nor are strings parsed as numbers. 

Because `Float64` is a concrete type and concrete types cannot be subclassed in Julia, such a definition can only be applied to arguments that are exactly of type `Float64`. 

It may often be useful, however, to write more general methods where the declared parameter types are abstract:

In [ ]:
function myfunc(x::Number, y::Number)
    2x + y
end

In [ ]:
myfunc(2.0, 3)

## Checking methods:
You can easily see how many methods exist for a function by entering the function object itself in an interactive session:

In [ ]:
myfunc

This output tells us that `myfunc` is a function object with two methods. To find out what the signatures of those methods are, use the `methods()` function:

In [ ]:
methods(myfunc)

## Exercise 4: Add a method

The function below adds any number to any other number.

The word add could have other meanings as well. Write a method of the add function that takes two strings and adds the
second to the end of the first.

So if called with "Hello " and "World!", the function returns "Hello World!".

Now create another method for the add function. This time the first argument is an array and the second is a number.

The function returns a new array with the number added to the end of the original array.

## Exercise 5: List methods

What methods does the add function have?

Check other functions you have used.

### Manually defined types

Combining multiple dispatch with manually defined types is really powerful:

In [ ]:
"Since MyInt is better than a standard Number, multiply it by 3"
function myfunc(x::Number, y::MyInt)
    2*x + 3*y.number
end

"Since MyInt is better than a standard Number, multiply it by 3"
function myfunc(x::MyInt, y::Number)
    result = 3*2*x.number + y
    
    # Return a MyInt type number
    MyInt(result)
end

In [ ]:
myfunc(5, MyInt(5))

In [ ]:
myfunc(MyInt(5), 5)

## Epidemic Simulation

We can extend almost any Julia function with custom behaviour for our custom datatypes.

Case in point, let's print the Cell type in a prettier way.

In [ ]:
# This is how we write a print function for a cell
function Base.show(io::IO, cell::Cell)
    if cell.status == infected
        print(io, "◼")
    else
        print(io, "◻")
    end
end

In [ ]:
# Create an infected and an uninfected cell
cell1 = Cell(uninfected, 0)
cell2 = Cell(infected, 0)

println(cell1, cell2)
interact!(cell1, cell2, 0.5)
println(cell1, cell2)

## Extra Exercise 1: Fruit Bowl

Write functions to keep track of fruit in your fruit bowl and to warn you when it's empty or if no more fruit will fit in. For simplicity, assume a given number of fruit will fit not matter the kind.

To get started, create a new type for a piece of fruit. The fruit can be either an apple, a banana or an orange. (You can add others based on your preference.)

You can use the @enum to represent types of fruit.

In [ ]:
@enum ...

Now create a mutable type called FruitBowl. It contains a list of fruit in the bowl and the maximum capacity of the bowl (integer).

In [ ]:
mutable struct FruitBowl
    ...
end

Now create an `add!`-function to add a piece of fruit to the bowl. Check that the bowl does not go over capacity.

Test your code by running the two cells below

In [ ]:
# Test it with an empty bowl
bowl = FruitBowl([], 4)
add!(bowl, apple)

In [ ]:
# Keep adding untill the bowl is full
add!(bowl, apple)
add!(bowl, banana)
add!(bowl, orange) 
add!(bowl, orange) # This one does not fit

# Extra:

## Macros
Finally, let's touch the metaprogramming capabilities of Julia. 

Since metaprogramming is a whole other topic (see bonus notebook) we will only cover the very basics of something you might encounter when dealing with Julia code: macros.

Macros provide a method to include generated code in the final body of a program. A macro maps a tuple of arguments to a returned **expression**, and the resulting expression is compiled directly.

This means that macros can change how functions work, hence the *meta* in metaprogramming.

In [ ]:
macro sayhello()
    return :( println("Hello, world!") )
end

## Macro invocation
Macros are invoked with the following general syntax:
```julia
@name expr1 expr2 ...
@name(expr1, expr2, ...)
```

In [ ]:
@sayhello

## Advanced: Tasks (aka Coroutines)
Tasks are a control flow feature that allows computations to be suspended and resumed in a flexible manner. More information can be found from the [documentation](https://docs.julialang.org/en/v1/manual/control-flow/#man-tasks-1)

In [ ]:
function producer(c::Channel)
    put!(c, "start")
    put!(c, 1)
    put!(c, 2)
    put!(c, 3)
    put!(c, 3)
    put!(c, "stop")
end;

chnl = Channel(producer);

In [ ]:
take!(chnl) # try executing me repeatedly